# Washington DOT Traffic API


In [30]:
import pandas as pd

import requests

## Import env variables


In [31]:
from dotenv import load_dotenv
import os

# load the .env file
load_dotenv()

# access the env variables
api_key = os.getenv("API_KEY")
travel_times_url = os.getenv("TRAVEL_TIMES_URL")
traffic_alert_url = os.getenv("TRAFFIC_ALERT_URL")
current_traffic_alerts_url = os.getenv("CURRENT_TRAFFIC_ALERTS_URL")

# print to verify
# print(f"API Key: {api_key}")
# print(f"Database URL: {db_url}")

In [ ]:
TRAVEL_TIMES_URL=http://wsdot.wa.gov/Traffic/api/TravelTimes/TravelTimesREST.svc/GetTravelTimesAsJson?AccessCode={ACCESSCODE}

TRAFFIC_ALERT_MAP_AREAS_URL=http://www.wsdot.wa.gov/Traffic/api/HighwayAlerts/HighwayAlertsREST.svc/GetMapAreasAsJson?AccessCode={ACCESSCODE}

CURRENT_TRAFFIC_ALERTS_URL=http://www.wsdot.wa.gov/Traffic/api/HighwayAlerts/HighwayAlertsREST.svc/GetAlertsAsJson?AccessCode={ACCESSCODE}

## Get API data


In [32]:
def get_data_from_api(url, access_key):
    print(url)
    url_api = url.format(ACCESSCODE=access_key)
    print(url_api)
    response = requests.get(url_api)

    # check if request was successful
    if response.status_code == 200:
        print("Data fetched successfully.")
        data = response.json()
        df = pd.DataFrame(data)
    else:
        print(f"Failed to fetch data. Status code: {response.status_code}")

    return df

In [33]:
df_travel_times = get_data_from_api(travel_times_url, api_key)
df.head()

http://wsdot.wa.gov/Traffic/api/TravelTimes/TravelTimesREST.svc/GetTravelTimesAsJson?AccessCode=
http://wsdot.wa.gov/Traffic/api/TravelTimes/TravelTimesREST.svc/GetTravelTimesAsJson?AccessCode=
Failed to fetch data. Status code: 400


UnboundLocalError: cannot access local variable 'df' where it is not associated with a value

In [10]:
df_traffic_alerts = get_data_from_api(traffic_alert_url, api_key)
df_traffic_alerts

Data fetched successfully.


,MapArea,MapAreaDescription
0,L2CE,Eastern
1,L2CL,Central
2,L2LO,Western
3,L2NE,Northeast
4,L2NN,Northwest
5,L2NW,North Central
6,L2PS,Puget Sound
7,L2SC,South Central
8,L2SE,Southeast
9,L2SW,Southwest


In [14]:
df_current_alerts = get_data_from_api(current_traffic_alerts_url, api_key)
df_current_alerts.head(100)

Data fetched successfully.


,AlertID,County,EndRoadwayLocation,EndTime,EventCategory,EventStatus,ExtendedDescription,HeadlineDescription,LastUpdatedTime,Priority,Region,StartRoadwayLocation,StartTime
0,624112,Grant,"{'Description': None, 'Direction': 'B', 'Latit...",None,Rest Area,Open,,The blue Lake rest area is closed for the winter.,/Date(1731629066113-0800)/,Low,North Central,"{'Description': None, 'Direction': 'B', 'Latit...",/Date(1731628800000-0800)/
1,626164,Spokane,"{'Description': None, 'Direction': 'B', 'Latit...",None,Road Report,Open,,"US195 from Rosalia to Spokane, road conditions...",/Date(1738020868367-0800)/,Low,Eastern,"{'Description': None, 'Direction': 'B', 'Latit...",/Date(1738020840000-0800)/
2,632850,None,"{'Description': None, 'Direction': 'B', 'Latit...",/Date(1738328400000-0800)/,Maintenance,Open,,"Nightly from 9 p.m. until 5 a.m. Sunday, Jan. ...",/Date(1737765776500-0800)/,High,Northwest,"{'Description': None, 'Direction': 'B', 'Latit...",/Date(1737765300000-0800)/
3,632245,None,"{'Description': None, 'Direction': 'B', 'Latit...",/Date(1738371600000-0800)/,Maintenance,Open,,"Weekdays, from 7 a.m. to 5 p.m., Wednesday, Ja...",/Date(1737148405100-0800)/,Low,Northwest,"{'Description': None, 'Direction': 'B', 'Latit...",/Date(1737148080000-0800)/
4,632782,None,"{'Description': None, 'Direction': 'S', 'Latit...",/Date(1738152000000-0800)/,Construction,Open,,"Tuesday, Jan. 27 to the morning of Wednesday, ...",/Date(1737738428410-0800)/,Medium,Northwest,"{'Description': None, 'Direction': 'S', 'Latit...",/Date(1737738900000-0800)/
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,634027,None,"{'Description': None, 'Direction': 'S', 'Latit...",/Date(1738326600000-0800)/,Construction,Open,,The southbound I-405 HOV off-ramp to southboun...,/Date(1738001089370-0800)/,Low,Northwest,"{'Description': None, 'Direction': 'S', 'Latit...",/Date(1737964800000-0800)/
96,626173,Ferry,"{'Description': None, 'Direction': 'B', 'Latit...",None,Pass Report,Open,,SR20 Sherman Pass conditions are: bare and dr...,/Date(1738022718913-0800)/,Low,Eastern,"{'Description': None, 'Direction': 'B', 'Latit...",/Date(1738022700000-0800)/
97,632021,None,"{'Description': None, 'Direction': 'N', 'Latit...",/Date(1738274400000-0800)/,Maintenance,Open,,"From 9 a.m. to 2 p.m. each day Monday, Jan. 27...",/Date(1736975754470-0800)/,Medium,Olympic,"{'Description': None, 'Direction': 'N', 'Latit...",/Date(1736975400000-0800)/
98,549502,None,"{'Description': None, 'Direction': 'B', 'Latit...",None,Closure,Open,,"Beginning June 15, 2023, until further notice:...",/Date(1730513039713-0700)/,Highest,Southwest,"{'Description': None, 'Direction': 'B', 'Latit...",/Date(1686869700000-0700)/
